# Getting Started with the Claude Agent API

Learn how to create and interact with stateful agent sessions using the Claude Agent API. This cookbook covers creating environments, launching sessions with agents, sending messages, and streaming responses.

> **Note:** The Claude Agent API is currently in beta. API signatures and features may change.

## Table of Contents

1. [Overview & Concepts](#overview)
2. [Setup & Installation](#setup)
3. [Creating an Environment](#environments)
4. [Creating Sessions](#sessions)
5. [Interacting with Sessions](#interacting)
6. [Managing Sessions](#managing)
7. [Best Practices](#best-practices)

## 1. Overview & Concepts {#overview}

### What is the Agent API?

The Agent API enables you to run AI agents in managed compute environments. Unlike the standard Messages API where you send prompts and receive responses, the Agent API provides:

- **Persistent compute environments** with configurable network access
- **Stateful agent sessions** that maintain conversation history
- **Built-in tools** for file operations, bash commands, and web access
- **Streaming responses** for real-time interaction

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Environment** | A compute environment where agents run. Defines container configuration and network access. |
| **Session** | A running agent instance within an environment, with a specific model, tools, and resources. |
| **Agent** | The Claude model configuration including system prompt and available tools. |
| **Events** | Messages exchanged between users and agents (user messages, agent responses, tool use). |

## 2. Setup & Installation {#setup}

### Prerequisites

- Python 3.8 or higher
- Anthropic API key with Agent API access from [console.anthropic.com](https://console.anthropic.com/)

In [ ]:
# Install dependencies
%pip install anthropic python-dotenv --quiet

In [1]:
import os
from datetime import datetime

import anthropic

# Verify SDK version (requires 0.75.0 or higher)
print(f"Anthropic SDK version: {anthropic.__version__}")

Anthropic SDK version: 0.75.0


### Configure API Key

Set your API key as an environment variable or load it from a `.env` file:

In [ ]:
# Option 1: Set directly (not recommended for production)
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# Option 2: Load from .env file
from dotenv import load_dotenv

load_dotenv()

# Initialize the client with beta headers
client = anthropic.Anthropic(
    default_headers={
        "anthropic-beta": "environments-2025-12-12,agent-api-2025-12-12",
    },
)

# Verify connection
if os.getenv("ANTHROPIC_API_KEY"):
    print("API key configured successfully")
    print(f"Base URL: {client.base_url}")
else:
    print("Warning: ANTHROPIC_API_KEY not found in environment")

## 3. Creating an Environment {#environments}

An environment defines the container configuration where your agents run. You typically create one environment and reuse it across many sessions.

### Environment Types

| Type | Description |
|------|-------------|
| `cloud` | Fully managed by Anthropic. Best for getting started quickly. |
| `self_hosted` | Self-hosted on your infrastructure (BYOC). See the BYOC cookbook. |

### Network Configurations

| Policy | Description |
|--------|-------------|
| `unrestricted` | Full internet access (use with caution) |
| `package_managers_and_custom` | Package manager URLs + your allowed hosts |
| `custom_only` | Only your specified allowed hosts |

In [3]:
import uuid

# Name prefix for environments created by this cookbook
# This helps identify and reuse environments in shared workspaces
ENV_NAME_PREFIX = "agent-api-cookbook-"


def get_or_create_environment():
    """
    Get an existing cookbook environment or create a new one.
    Only looks for environments with the cookbook's name prefix to avoid
    accidentally using environments created by others in shared workspaces.
    """
    # Look for an existing environment created by this cookbook
    existing = client.beta.environments.list()
    for env in existing.data:
        if env.name.startswith(ENV_NAME_PREFIX):
            print(f"Using existing cookbook environment: {env.id}")
            print(f"  Name: {env.name}")
            return env

    # Create a new environment with unrestricted networking
    print("Creating new environment...")
    environment = client.beta.environments.create(
        name=f"{ENV_NAME_PREFIX}{uuid.uuid4().hex[:8]}",
        description="Environment for Agent API cookbook",
        config={
            "type": "cloud",
            "networking": {"type": "unrestricted"},
        },
        metadata={
            "purpose": "cookbook-demo",
        },
    )

    print(f"Created environment: {environment.id}")
    print(f"  Name: {environment.name}")
    print(f"  State: {environment.state}")
    return environment


environment = get_or_create_environment()

Using existing cookbook environment: env_01JUiZocETNvu2rukynNeqwZ
  Name: agent-api-cookbook-restricted-870f5f67


### Listing Cookbook Environments

View environments created by this cookbook:

In [4]:
# List environments created by this cookbook
all_environments = client.beta.environments.list()
cookbook_environments = [env for env in all_environments.data if env.name.startswith(ENV_NAME_PREFIX)]

print(f"Cookbook Environments (prefix: '{ENV_NAME_PREFIX}'):")
print("=" * 70)

if not cookbook_environments:
    print("\nNo cookbook environments found. Run the cell above to create one.")
else:
    for env in cookbook_environments:
        print(f"\nID: {env.id}")
        print(f"  Name: {env.name}")
        print(f"  State: {env.state}")
        print(f"  Type: {env.config.type if hasattr(env.config, 'type') else 'unknown'}")

    print("\n" + "=" * 70)
    print(f"Total: {len(cookbook_environments)} cookbook environment(s)")

Cookbook Environments (prefix: 'agent-api-cookbook-'):

ID: env_01JUiZocETNvu2rukynNeqwZ
  Name: agent-api-cookbook-restricted-870f5f67
  State: active
  Type: cloud

ID: env_01NMrtnuEMed2pzteakZcvg3
  Name: agent-api-cookbook-83803e20
  State: active
  Type: cloud

Total: 2 cookbook environment(s)


### Creating an Environment with Restricted Networking

For more secure environments, you can restrict network access:

In [5]:
# Example: Create an environment with restricted networking

restricted_env = client.beta.environments.create(
    name=f"{ENV_NAME_PREFIX}restricted-{uuid.uuid4().hex[:8]}",
    description="Environment with package managers and custom hosts only",
    config={
        "type": "cloud",
        "networking": {
            "type": "package_managers_and_custom",
            "allowed_hosts": ["api.github.com", "your-api.example.com"],
        },
    },
)
print(f"Created restricted environment: {restricted_env.id}")

Created restricted environment: env_01JUiZocETNvu2rukynNeqwZ


## 4. Creating Sessions {#sessions}

A session represents a stateful interaction with an agent. When you create a session, you specify:
- The environment where it runs
- The agent configuration (model, system prompt, tools)
- Optional resources (GitHub repos, files)

### Available Agent Tools

| Tool | Type | Description |
|------|------|-------------|
| Bash | `agent_bash_20251212` | Execute shell commands |
| Read | `agent_read_20251212` | Read file contents |
| Write | `agent_write_20251212` | Write to files |
| Edit | `agent_edit_20251212` | Edit existing files |
| Grep | `agent_grep_20251212` | Search file contents |
| Glob | `agent_glob_20251212` | Find files by pattern |
| Web Fetch | `agent_web_fetch_20251212` | Fetch web content |
| Web Search | `agent_web_search_20251212` | Search the web |

In [5]:
def get_or_create_session(environment_id: str):
    """
    Get an existing session in the cookbook environment or create a new one.
    Only reuses sessions that are running or idle (not closed/archived).
    """
    # Look for an existing active session in this environment
    existing_sessions = client.beta.sessions.list(limit=50)
    for s in existing_sessions.data:
        if s.environment == environment_id and s.status in ("running", "idle"):
            print(f"Using existing session: {s.id}")
            print(f"  Title: {s.title}")
            print(f"  Status: {s.status}")
            return s

    # Create a new session with basic tools
    print("Creating new session...")
    session = client.beta.sessions.create(
        environment=environment_id,
        agent={
            "model": "claude-sonnet-4-5-20250929",
            "system": "You are a helpful coding assistant. Help users with programming tasks.",
            "tools": [
                {"type": "agent_bash_20251212", "name": "agent_bash"},
                {"type": "agent_read_20251212", "name": "agent_read"},
                {"type": "agent_write_20251212", "name": "agent_write"},
                {"type": "agent_grep_20251212", "name": "agent_grep"},
            ],
        },
        title="Coding Assistant Session",
    )

    print(f"Created session: {session.id}")
    print(f"  Status: {session.status}")
    return session


session = get_or_create_session(environment.id)

Using existing session: session_01MirMuz34eCP9p7TWxtEfMA
  Title: Code Review Session
  Status: running


### Using the Agent Toolset

Instead of enabling individual tools, you can use `agent_toolset_20251212` to enable all tools at once with optional overrides:

In [6]:
# Create a session with all tools enabled via toolset

toolset_session = client.beta.sessions.create(
    environment=environment.id,
    agent={
        "model": "claude-sonnet-4-5-20250929",
        "system": "You are a helpful assistant.",
        "tools": [
            {
                "type": "agent_toolset_20251212",
                "default_config": {"enabled": True},
                # Optionally disable specific tools:
                "configs": [
                    {"name": "agent_web_fetch", "enabled": False},
                ],
            },
        ],
    },
    title="Full Toolset Session",
)
print(f"Created toolset session: {toolset_session.id}")

Created toolset session: session_01RMjXddyMVo2Vcv1RzLGdou


### Session with GitHub Repository

You can mount a GitHub repository into a session, allowing the agent to read and modify code:

In [ ]:
# Create a session with a GitHub repository
# Note: Set GITHUB_TOKEN environment variable with your token

github_token = os.getenv("GITHUB_TOKEN")

if not github_token:
    print("Warning: GITHUB_TOKEN not set. Skipping GitHub repository example.")
else:
    repo_session = client.beta.sessions.create(
        environment=environment.id,
        agent={
            "model": "claude-sonnet-4-5-20250929",
            "system": "You are a code review assistant. Analyze code and suggest improvements.",
            "tools": [
                {"type": "agent_bash_20251212", "name": "agent_bash"},
                {"type": "agent_read_20251212", "name": "agent_read"},
                {"type": "agent_grep_20251212", "name": "agent_grep"},
            ],
        },
        resources=[
            {
                "type": "github_repository",
                "url": "https://github.com/your-org/your-repo",  # Replace with your repo
                "authorization_token": github_token,
                "checkout": {
                    "type": "branch",
                    "name": "main",
                },
                "mount_path": "/workspace/repo",
            }
        ],
        title="Code Review Session",
    )
    print(f"Created repo session: {repo_session.id}")

## 5. Interacting with Sessions {#interacting}

### Sending Messages and Streaming Responses

The pattern for interacting with a session is:
1. Start listening to the stream (before sending)
2. Send a user message event
3. Process streaming events as they arrive

In [6]:
def chat(session_id: str, message: str):
    """
    Send a message to a session and stream the response.

    Event types you'll receive:
      - agent: Agent response with text, tool use, or thinking content
      - status_idle: Agent turn complete (break the loop here)
      - status_running: Session is processing
      - error: An error occurred
    """
    print(f"\nYou: {message}")
    print("\nAssistant: ", end="", flush=True)

    # Start the stream and send the message
    with client.beta.sessions.stream(session_id=session_id) as stream:
        client.beta.sessions.events.send(
            session_id=session_id,
            events=[
                {
                    "type": "user",
                    "content": [{"type": "text", "text": message}],
                },
            ],
        )

        # Process streaming events
        for event in stream:
            if event.type == "agent":
                for content_block in event.content:
                    if hasattr(content_block, "text"):
                        print(content_block.text, end="", flush=True)
                    elif hasattr(content_block, "name"):
                        # Tool use
                        print(f"\n[Using tool: {content_block.name}]")
            elif event.type == "status_idle":
                # Agent turn complete - exit the stream
                break
            elif event.type == "error":
                print(f"\n[Error: {event.error.message if event.error else 'Unknown error'}]")
                break

    print("\n")

In [7]:
# Try chatting with the session
chat(session.id, "What tools do you have access to?")


You: What tools do you have access to?

Assistant: I have access to the following tools:

## **Bash**
Executes terminal commands in a persistent shell session. Used for:
- Git operations (status, commit, push, pull, etc.)
- Package management (npm, pip, etc.)
- Docker and other system commands
- GitHub CLI operations (gh commands)
- General shell operations

**Note:** This tool is for terminal operations, not file operations. I avoid using bash commands like `cat`, `grep`, `find`, etc., in favor of specialized tools.

## **Grep**
A powerful search tool built on ripgrep for finding patterns in code and files. Features:
- Full regex syntax support
- File filtering by type or glob patterns
- Multiple output modes (content with line numbers, file names only, or counts)
- Case-insensitive search option
- Context lines (show lines before/after matches)
- Multiline pattern matching

## **Read**
Reads files from the filesystem. Capabilities:
- Read any file with an absolute path
- Handle larg

In [8]:
# Continue the conversation - the session maintains state
chat(session.id, "Create a file called hello.py with a simple greeting function.")


You: Create a file called hello.py with a simple greeting function.

Assistant: I'll create a simple Python file with a greeting function for you.
[Using tool: agent_bash]

[Using tool: agent_read]
Perfect! I've created `hello.py` with a simple greeting function. The file includes:

- **`greet(name)`** - A function that takes a name and returns a personalized greeting message
- **Docstring** - Documentation explaining the function's purpose, parameters, and return value
- **Example usage** - Demo code that runs when you execute the script directly

You can run it with:
```bash
python hello.py
```

This will output:
```
Hello, World! Welcome!
Hello, Python! Welcome!
```

Or you can import and use the `greet()` function in other Python scripts!I see that there are untracked files in the repository (the `hello.py` file we just created). This is just a notification from your git hook.

If you'd like me to commit and push these changes, just let me know! Otherwise, the file is ready to use

In [9]:
# Ask about the file we just created
chat(session.id, "Read the contents of hello.py and explain what it does.")


You: Read the contents of hello.py and explain what it does.

Assistant: 
[Using tool: agent_read]
## Explanation of hello.py

This Python file contains a simple greeting function with the following components:

### **The `greet()` function (lines 1-11)**
- **Purpose**: Takes a person's name as input and returns a personalized greeting message
- **Parameter**: `name` - the name of the person to greet
- **Return value**: A formatted string like "Hello, World! Welcome!"
- **Implementation**: Uses an f-string (formatted string literal) to insert the provided name into the greeting message
- **Documentation**: Includes a docstring that explains the function's purpose, parameters, and return value

### **The main execution block (lines 14-17)**
- **Purpose**: Demonstrates how to use the `greet()` function
- **Condition**: `if __name__ == "__main__"` ensures this code only runs when the script is executed directly (not when imported as a module)
- **Behavior**: Calls the `greet()` function 

### Viewing Session Events

You can retrieve all events from a session to see the full conversation history:

In [12]:
def list_events(session_id: str, limit: int = 30):
    """List events from a session."""
    response = client.beta.sessions.events.list(session_id=session_id, limit=limit)

    print(f"Found {len(response.data)} events:")
    for event in response.data:
        print(f"\nEvent type: {event.type}")
        if hasattr(event, "content"):
            for block in event.content[:1]:  # Show first content block
                if hasattr(block, "text"):
                    preview = block.text[:100] + "..." if len(block.text) > 100 else block.text
                    print(f"  Content: {preview}")


list_events(session.id)

Found 30 events:

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_pending

Event type: status_running

Event type: user
  Content: What tools do you have access to?

Event type: agent

Event type: agent
  Content: I have access to the following tools:

## **Bash**
Executes terminal commands in a persistent shell ...

Event type: status_idle

Event type: status_running

Event type: user
  Content: Create a file called hello.py with a simple greeting function.

Event type: agent

Event type: agent
  Content: I'll create a simple Python file with a greeting function for you.

Event type: agent

Event type: agent

Event type: agent

Event type: agent

Event type: agent

Event type: agent

Event type: agent
  Content: Perfect! I've created `hello.py` with a simple greetin

### Listing Sessions

In [13]:
# List sessions for cookbook environments
# First, get cookbook environment IDs
all_environments = client.beta.environments.list()
cookbook_env_ids = {
    env.id for env in all_environments.data 
    if env.name.startswith(ENV_NAME_PREFIX)
}

# List all sessions and filter by cookbook environments
all_sessions = client.beta.sessions.list(limit=20)
cookbook_sessions = [
    s for s in all_sessions.data 
    if s.environment in cookbook_env_ids
]

print(f"Sessions in Cookbook Environments (prefix: '{ENV_NAME_PREFIX}'):")
print("=" * 70)

if not cookbook_sessions:
    print("\nNo sessions found in cookbook environments.")
else:
    for s in cookbook_sessions:
        print(f"\nID: {s.id}")
        print(f"  Title: {getattr(s, 'title', 'Untitled')}")
        print(f"  Status: {s.status}")
        print(f"  Environment: {s.environment}")
        print(f"  Created: {s.created_at}")

    print("\n" + "=" * 70)
    print(f"Total: {len(cookbook_sessions)} session(s) in cookbook environments")

Sessions in Cookbook Environments (prefix: 'agent-api-cookbook-'):

ID: session_01MirMuz34eCP9p7TWxtEfMA
  Title: Code Review Session
  Status: idle
  Environment: env_01JUiZocETNvu2rukynNeqwZ
  Created: 2026-01-19 02:27:09.631633+00:00

ID: session_01RMjXddyMVo2Vcv1RzLGdou
  Title: Full Toolset Session
  Status: running
  Environment: env_01JUiZocETNvu2rukynNeqwZ
  Created: 2026-01-19 02:21:44.771407+00:00

ID: session_01Ee6PZfWHa87qu83VtZjaxg
  Title: Coding Assistant Session
  Status: idle
  Environment: env_01JUiZocETNvu2rukynNeqwZ
  Created: 2026-01-19 02:20:19.917395+00:00

Total: 3 session(s) in cookbook environments


### Retrieving Session Details

In [14]:
# Retrieve a specific session
session_details = client.beta.sessions.retrieve(session_id=session.id)

print(f"Session: {session_details.id}")
print(f"  Title: {session_details.title}")
print(f"  Status: {session_details.status}")
print(f"  Created: {session_details.created_at}")
print(f"  Updated: {session_details.updated_at}")

Session: session_01MirMuz34eCP9p7TWxtEfMA
  Title: Code Review Session
  Status: idle
  Created: 2026-01-19 02:27:09.631633+00:00
  Updated: 2026-01-19 02:54:49.172059+00:00


### Updating Sessions

In [15]:
# Update session title
updated_session = client.beta.sessions.update(
    session_id=session.id,
    title=f"Updated: Coding Assistant - {datetime.now().strftime('%Y-%m-%d')}",
)

print(f"Updated session title: {updated_session.title}")

Updated session title: Updated: Coding Assistant - 2026-01-19


### Archiving Sessions

When you're done with a session, you can archive it:

In [18]:
# Archive the session when done

archived = client.beta.sessions.archive(session_id=session.id)
print(f"Archived session: {archived.id}")

Archived session: session_01Ee6PZfWHa87qu83VtZjaxg


### Managing Environments

In [19]:
# List cookbook environments
all_environments = client.beta.environments.list()
cookbook_environments = [env for env in all_environments.data if env.name.startswith(ENV_NAME_PREFIX)]

print(f"Cookbook Environments (prefix: '{ENV_NAME_PREFIX}'):")
print("=" * 60)

if not cookbook_environments:
    print("\nNo cookbook environments found.")
else:
    for env in cookbook_environments:
        print(f"\nID: {env.id}")
        print(f"  Name: {env.name}")
        print(f"  State: {env.state}")
        print(f"  Type: {env.config.type if hasattr(env.config, 'type') else 'unknown'}")

    print("\n" + "=" * 60)
    print(f"Total: {len(cookbook_environments)} cookbook environment(s)")

Cookbook Environments (prefix: 'agent-api-cookbook-'):

ID: env_01JUiZocETNvu2rukynNeqwZ
  Name: agent-api-cookbook-restricted-870f5f67
  State: active
  Type: cloud

ID: env_01NMrtnuEMed2pzteakZcvg3
  Name: agent-api-cookbook-83803e20
  State: active
  Type: cloud

Total: 2 cookbook environment(s)


In [20]:
# Update environment metadata
updated_env = client.beta.environments.update(
    environment_id=environment.id,
    description="Updated: Agent API cookbook environment",
    metadata={
        "purpose": "cookbook-demo",
        "updated_at": datetime.now().isoformat(),
    },
)

print(f"Updated environment: {updated_env.name}")
print(f"New description: {updated_env.description}")

Updated environment: agent-api-cookbook-restricted-870f5f67
New description: Updated: Agent API cookbook environment


## 7. Best Practices {#best-practices}

### Security

1. **Use restrictive networking** - Start with `custom_only` or `package_managers_and_custom` and add hosts as needed
2. **Rotate tokens** - Regularly rotate GitHub tokens and API keys
3. **Scope appropriately** - Use `account` scope for sensitive workloads

### Performance

1. **Choose the right model** - Use Haiku for simple tasks, Sonnet for everyday coding, and Opus for complex multi-step tasks
2. **Reuse environments** - Create one environment and reuse it across sessions
3. **Archive unused sessions** - Clean up sessions when done to free resources

### Error Handling

1. **Check session status** - Verify session is `running` before sending messages
2. **Handle stream disconnects** - Implement reconnection logic for long-running sessions
3. **Monitor events** - Use the events list API to debug issues

### Common Issues

| Issue | Solution |
|-------|----------|
| Environment creation fails | Check API key has Agent API access |
| Session stuck in pending | Verify environment is active |
| GitHub resource fails | Ensure token has required repository permissions |
| Network errors | Check networking policy allows required hosts |
| Name conflict | Use unique names (UUID suffix recommended) |

## Cleanup

Clean up resources created in this cookbook:

In [22]:
# Uncomment to clean up resources:

# Archive the session
# client.beta.sessions.archive(session_id=session.id)
# print(f"Archived session: {session.id}")

# Archive the environment (optional - you may want to reuse it)
client.beta.environments.archive(environment_id="env_01NMrtnuEMed2pzteakZcvg3")
print(f"Archived environment: {environment.id}")

Archived environment: env_01JUiZocETNvu2rukynNeqwZ


## Next Steps

Now that you understand the Agent API basics:

1. **Explore BYOC** - For enterprise needs, see the [BYOC Environments cookbook](./byoc_environments.ipynb)
2. **Add GitHub integration** - Mount repositories and let agents work with your code
3. **Build custom tools** - Use MCP servers to add custom capabilities

### Resources

- [Anthropic API Documentation](https://docs.anthropic.com/)
- [Anthropic Console](https://console.anthropic.com/)